In [39]:
import carla
import math
import random
import time
import numpy as np
import cv2

In [40]:
client = carla.Client('192.168.5.145', 2000)
# client.get_available_maps()

In [41]:
world = client.load_world('Town01')

In [42]:
bp_lib = world.get_blueprint_library()
world_map = world.get_map()
spawn_points = world_map.get_spawn_points()

In [43]:
vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020')
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

In [44]:
spectator = world.get_spectator()
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4, z=2.5)),
                            vehicle.get_transform().rotation)
spectator.set_transform(transform)

In [45]:
world.set_weather(carla.WeatherParameters.ClearSunset)

In [46]:
settings = world.get_settings()
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.05

world.apply_settings(settings)

521

In [47]:
traffic_manager = client.get_trafficmanager()
traffic_manager.set_synchronous_mode(True)

In [48]:
spawn_points = world.get_map().get_spawn_points()

In [34]:
for i, spawn_point in enumerate(spawn_points):
    world.debug.draw_string(spawn_point.location, str(i), life_time=10)

# In synchronous mode, we need to run the simulation to fly the spectator
while True:
    world.tick()

KeyboardInterrupt: 

In [49]:
# Route 1
spawn_point_1 = spawn_points[32]
# Create route 1 from the chosen spawn points
route_1_indices = [129, 28, 124, 33, 97, 119, 58, 154, 147]
route_1 = []
for ind in route_1_indices:
    route_1.append(spawn_points[ind].location)

# Route 2
spawn_point_2 = spawn_points[149]
# Create route 2 from the chosen spawn points
route_2_indices = [21, 76, 38, 34, 90, 3]
route_2 = []
for ind in route_2_indices:
    route_2.append(spawn_points[ind].location)

# Now let's print them in the map so we can see our routes
world.debug.draw_string(spawn_point_1.location, 'Spawn point 1', life_time=30, color=carla.Color(255, 0, 0))
world.debug.draw_string(spawn_point_2.location, 'Spawn point 2', life_time=30, color=carla.Color(0, 0, 255))

for ind in route_1_indices:
    spawn_points[ind].location
    world.debug.draw_string(spawn_points[ind].location, str(ind), life_time=60, color=carla.Color(255, 0, 0))

for ind in route_2_indices:
    spawn_points[ind].location
    world.debug.draw_string(spawn_points[ind].location, str(ind), life_time=60, color=carla.Color(0, 0, 255))

while True:
    world.tick()

KeyboardInterrupt: 

In [37]:
# Set delay to create gap between spawn times
spawn_delay = 20
counter = spawn_delay

# Set max vehicles (set smaller for low hardward spec)
max_vehicles = 200
# Alternate between spawn points
alt = False

while True:
    world.tick()

    n_vehicles = len(world.get_actors().filter('*vehicle*'))
    # vehicle_bp = random.choice(blueprints)

    # Spawn vehicle only after delay
    if counter == spawn_delay and n_vehicles < max_vehicles:
        # Alternate spawn points
        if alt:
            vehicle = world.try_spawn_actor(vehicle_bp, spawn_point_1)
        else:
            vehicle = world.try_spawn_actor(vehicle_bp, spawn_point_2)

        if vehicle:  # IF vehicle is succesfully spawned
            vehicle.set_autopilot(True)  # Give TM control over vehicle

            # Set parameters of TM vehicle control, we don't want lane changes
            traffic_manager.update_vehicle_lights(vehicle, True)
            traffic_manager.random_left_lanechange_percentage(vehicle, 0)
            traffic_manager.random_right_lanechange_percentage(vehicle, 0)
            traffic_manager.auto_lane_change(vehicle, False)

            # Alternate between routes
            if alt:
                traffic_manager.set_path(vehicle, route_1)
                alt = False
            else:
                traffic_manager.set_path(vehicle, route_2)
                alt = True

            vehicle = None

        counter -= 1
    elif counter > 0:
        counter -= 1
    elif counter == 0:
        counter = spawn_delay


KeyboardInterrupt: 

In [ ]:
vehicle.set_autopilot(True)

# Set parameters of TM vehicle control, we don't want lane changes
traffic_manager.update_vehicle_lights(vehicle, True)
traffic_manager.random_left_lanechange_percentage(vehicle, 0)
traffic_manager.random_right_lanechange_percentage(vehicle, 0)
traffic_manager.auto_lane_change(vehicle, False)

traffic_manager.set_path(vehicle, route_1)

In [ ]:
while True:
    world.tick()